In [1]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from datetime import datetime

Population data : https://www.ers.usda.gov/data-products/county-level-data-sets/

## 1. Data Loading and Cleaning

### 1A. Load data about Mask use survey by New York Times

In [2]:
url = "https://raw.githubusercontent.com/nytimes/covid-19-data/master/mask-use/mask-use-by-county.csv"
maskdf = pd.read_csv(url)
maskdf = maskdf.rename(columns = {"COUNTYFP":"fips", "NEVER":"Mask-Never", "RARELY":"Mask-Rarely", "SOMETIMES":"Mask-Sometimes", "FREQUENTLY":"Mask-Frequently", "ALWAYS":"Mask-Always"}) 
maskdf.head()

,fips,Mask-Never,Mask-Rarely,Mask-Sometimes,Mask-Frequently,Mask-Always
0,1001,0.053,0.074,0.134,0.295,0.444
1,1003,0.083,0.059,0.098,0.323,0.436
2,1005,0.067,0.121,0.120,0.201,0.491
3,1007,0.020,0.034,0.096,0.278,0.572
4,1009,0.053,0.114,0.180,0.194,0.459


In [14]:
maskdf.to_csv('C:/Users/ysman/Documents/springboard/1 Google Mobility and Covid/python df exports/mask.csv', sep=',', index=True)

### 1B . Get Google Mobility Data

In [3]:
urlmobil = "C:/Users/ysman/Documents/springboard/1 Google Mobility and Covid/Google_Mobility_Report_usa.csv"
mobility =  pd.read_csv(urlmobil)
mobility = mobility.rename(columns = {"date":"Date", "census_fips_code":"fips", "country_region":"country", "sub_region_1":"state","sub_region_2":"county",
                                      "retail_and_recreation_percent_change_from_baseline":"recreation",
                                      "grocery_and_pharmacy_percent_change_from_baseline":"grocery" ,
                                      "parks_percent_change_from_baseline":"parks","transit_stations_percent_change_from_baseline":"transit", 
                                      "workplaces_percent_change_from_baseline":"workspace",
                                      "residential_percent_change_from_baseline":"residential" }) 
mobility.drop(["country_region_code", "metro_area", "iso_3166_2_code"], axis = 1, inplace = True) 
mobility = mobility[mobility['state'].notna()]
mobility = mobility[mobility['fips'].notna()]

converttype = {"fips":int}
mobility = mobility.astype(converttype)

### 1C. Merge Google Mobility and Mask Use Data

In [4]:
mobility = pd.merge(mobility,  
                     maskdf,  
                     on ='fips',  
                     how ='left')
mobility["Date"]= pd.to_datetime(mobility["Date"])

In [5]:
countydf = pd.read_csv('C:/Users/ysman/Documents/springboard/1 Google Mobility and Covid/county fips.csv')
countydf = countydf.rename(columns = {"FIPS":"fips", "Name":"County"})
countydf

,fips,County,State
0,1001,Autauga,AL
1,1003,Baldwin,AL
2,1005,Barbour,AL
3,1007,Bibb,AL
4,1009,Blount,AL
...,...,...,...
3227,72151,Yabucoa,PR
3228,72153,Yauco,PR
3229,78010,St. Croix,VI
3230,78020,St. John,VI


In [6]:
mobility = pd.merge(mobility,  
                     countydf,  
                     on ='fips',  
                     how ='left')

In [7]:
mobility.drop(["State"], axis = 1, inplace = True) 
mobility

,country,state,county,fips,Date,recreation,grocery,parks,transit,workspace,residential,Mask-Never,Mask-Rarely,Mask-Sometimes,Mask-Frequently,Mask-Always,County
0,United States,Alabama,Autauga County,1001,2020-02-15,5.0,7.0,NaN,NaN,-4.0,NaN,0.053,0.074,0.134,0.295,0.444,Autauga
1,United States,Alabama,Autauga County,1001,2020-02-16,0.0,1.0,-23.0,NaN,-4.0,NaN,0.053,0.074,0.134,0.295,0.444,Autauga
2,United States,Alabama,Autauga County,1001,2020-02-17,8.0,0.0,NaN,NaN,-27.0,5.0,0.053,0.074,0.134,0.295,0.444,Autauga
3,United States,Alabama,Autauga County,1001,2020-02-18,-2.0,0.0,NaN,NaN,2.0,0.0,0.053,0.074,0.134,0.295,0.444,Autauga
4,United States,Alabama,Autauga County,1001,2020-02-19,-2.0,0.0,NaN,NaN,2.0,0.0,0.053,0.074,0.134,0.295,0.444,Autauga
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564820,United States,Wyoming,Weston County,56045,2020-09-14,NaN,NaN,NaN,NaN,-13.0,NaN,0.142,0.129,0.148,0.207,0.374,Weston
564821,United States,Wyoming,Weston County,56045,2020-09-15,NaN,NaN,NaN,NaN,-11.0,NaN,0.142,0.129,0.148,0.207,0.374,Weston
564822,United States,Wyoming,Weston County,56045,2020-09-16,NaN,NaN,NaN,NaN,-12.0,NaN,0.142,0.129,0.148,0.207,0.374,Weston
564823,United States,Wyoming,Weston County,56045,2020-09-17,NaN,NaN,NaN,NaN,-7.0,NaN,0.142,0.129,0.148,0.207,0.374,Weston


In [8]:
mobility_statelevel = mobility.drop('County', axis =1)
mobility_statelevel = mobility.drop('county', axis =1)

In [9]:
mobility_statelevel = pd.DataFrame(mobility_statelevel.groupby(['state','Date']).agg({'recreation':'median','grocery':'median','parks':'median','transit':'median', 'workspace':'median', 'residential':'median','Mask-Never':'median','Mask-Rarely':'median', 'Mask-Sometimes':'median' ,'Mask-Frequently':'median','Mask-Always':'median'})).reset_index()
mobility_statelevel

,state,Date,recreation,grocery,parks,transit,workspace,residential,Mask-Never,Mask-Rarely,Mask-Sometimes,Mask-Frequently,Mask-Always
0,Alabama,2020-02-15,6.0,1.0,55.0,5.0,2.0,-1.0,0.0775,0.0735,0.1355,0.2005,0.4675
1,Alabama,2020-02-16,-0.5,-3.0,-2.5,4.0,-2.0,0.0,0.0780,0.0730,0.1350,0.2010,0.4690
2,Alabama,2020-02-17,1.0,-3.0,10.0,8.0,-15.0,4.0,0.0770,0.0740,0.1360,0.2010,0.4660
3,Alabama,2020-02-18,-4.0,-6.0,-13.5,0.0,1.0,2.0,0.0770,0.0740,0.1360,0.2010,0.4660
4,Alabama,2020-02-19,2.0,0.0,3.0,5.0,2.0,0.0,0.0770,0.0740,0.1360,0.2010,0.4660
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11113,Wyoming,2020-09-15,NaN,NaN,NaN,NaN,-15.0,0.5,0.1460,0.1540,0.1205,0.2220,0.3295
11114,Wyoming,2020-09-16,NaN,NaN,NaN,NaN,-16.5,1.0,0.1460,0.1540,0.1205,0.2220,0.3295
11115,Wyoming,2020-09-17,NaN,NaN,NaN,NaN,-16.0,1.0,0.1450,0.1530,0.1210,0.2210,0.3350
11116,Wyoming,2020-09-18,NaN,NaN,NaN,NaN,-15.5,0.5,0.1460,0.1540,0.1205,0.2220,0.3295


### 1D. Government interventions by date

In [10]:
urldaily = "https://raw.githubusercontent.com/OxCGRT/USA-covid-policy/master/data/OxCGRT_US_latest.csv"
daily = pd.read_csv(urldaily)
daily["Date"] = pd.to_datetime(daily["Date"], format='%Y%m%d')
daily = daily[daily['RegionName'].notna()]
daily = daily[daily['Jurisdiction'] == 'STATE_ALL']
daily = daily.rename(columns={"RegionName":"state"})

In [11]:
daily.drop(['CountryName','CountryCode','RegionCode','Jurisdiction','C1_Notes','C2_Notes','C3_Notes','C4_Notes','C5_Notes','C6_Notes','C7_Notes','C8_Notes','E1_Notes',
 'E2_Notes','E3_Notes','E4_Notes','H1_Notes','H2_Notes','H3_Notes','H4_Notes','H5_Notes','M1_Wildcard' ,'M1_Notes','ConfirmedCases','ConfirmedDeaths', 'StringencyIndex',
 'StringencyIndexForDisplay','StringencyLegacyIndex','StringencyLegacyIndexForDisplay','GovernmentResponseIndex',
 'GovernmentResponseIndexForDisplay','ContainmentHealthIndex','ContainmentHealthIndexForDisplay','EconomicSupportIndex',
 'EconomicSupportIndexForDisplay'], axis = 1, inplace = True)  

In [12]:
list(daily.columns)

['state',
 'Date',
 'C1_School closing',
 'C1_Flag',
 'C2_Workplace closing',
 'C2_Flag',
 'C3_Cancel public events',
 'C3_Flag',
 'C4_Restrictions on gatherings',
 'C4_Flag',
 'C5_Close public transport',
 'C5_Flag',
 'C6_Stay at home requirements',
 'C6_Flag',
 'C7_Restrictions on internal movement',
 'C7_Flag',
 'C8_International travel controls',
 'E1_Income support',
 'E1_Flag',
 'E2_Debt/contract relief',
 'E3_Fiscal measures',
 'E4_International support',
 'H1_Public information campaigns',
 'H1_Flag',
 'H2_Testing policy',
 'H3_Contact tracing',
 'H4_Emergency investment in healthcare',
 'H5_Investment in vaccines']

In [22]:
daily

,state,Date,C1_School closing,C1_Flag,C2_Workplace closing,C2_Flag,C3_Cancel public events,C3_Flag,C4_Restrictions on gatherings,C4_Flag,...,E1_Flag,E2_Debt/contract relief,E3_Fiscal measures,E4_International support,H1_Public information campaigns,H1_Flag,H2_Testing policy,H3_Contact tracing,H4_Emergency investment in healthcare,H5_Investment in vaccines
576,Alaska,2020-01-01,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,...,NaN,0.0,NaN,NaN,0.0,NaN,0.0,0.0,NaN,NaN
577,Alaska,2020-01-02,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,...,NaN,0.0,NaN,NaN,0.0,NaN,0.0,0.0,NaN,NaN
578,Alaska,2020-01-03,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,...,NaN,0.0,NaN,NaN,0.0,NaN,0.0,0.0,NaN,NaN
579,Alaska,2020-01-04,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,...,NaN,0.0,NaN,NaN,0.0,NaN,0.0,0.0,NaN,NaN
580,Alaska,2020-01-05,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,...,NaN,0.0,NaN,NaN,0.0,NaN,0.0,0.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15547,Wyoming,2020-10-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15548,Wyoming,2020-10-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15549,Wyoming,2020-10-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15550,Wyoming,2020-10-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Export CSV files

In [13]:
mobility.to_csv('C:/Users/ysman/Documents/springboard/1 Google Mobility and Covid/python df exports/mobiliy_Countylevel.csv', sep=',',index=True)
mobility_statelevel.to_csv('C:/Users/ysman/Documents/springboard/1 Google Mobility and Covid/python df exports/mobiliy_Statelevel.csv', sep=',',index=True)

In [23]:
daily.to_csv('C:/Users/ysman/Documents/springboard/1 Google Mobility and Covid/python df exports/intervention_Statelevel.csv', sep=',',index=True)